## Multiple Parallel Series
        -- An alternate time series problem is the case where there are multiple parallel time series and a value  must be predicted for each.

In [1]:
################### Function to splitting a multivariate time series dataset into a supervised learning problem#######

In [20]:
from numpy import array
from numpy import hstack
from keras.models import Sequential, Model
from keras.layers import Dense, Input

In [21]:
def split_sequences(sequences, n_steps): 
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the dataset
        if end_ix > len(sequences)-1: 
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :] 
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps) 
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])


(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


### Multiple Parallel series input and Vector-output MLP Model

In [22]:
X, y = split_sequences(dataset, n_steps)
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
n_output = y.shape[1]
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input)) 
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')
print(model.summary())
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]]) 
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 303       
Total params: 1,303
Trainable params: 1,303
Non-trainable params: 0
_________________________________________________________________
None
[[100.0369   104.982635 205.2427  ]]


### Multiple Parallel series input and Multi-output MLP Model

In [23]:

X, y = split_sequences(dataset, n_steps)
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
# separate output
y1 = y[:, 0].reshape((y.shape[0], 1)) 
y2 = y[:, 1].reshape((y.shape[0], 1)) 
y3 = y[:, 2].reshape((y.shape[0], 1)) # define model
visible = Input(shape=(n_input,))
dense = Dense(100, activation='relu')(visible)
# define output 1
output1 = Dense(1)(dense)
# define output 2
output2 = Dense(1)(dense)
# define output 2
output3 = Dense(1)(dense)
# tie together
model = Model(inputs=visible, outputs=[output1, output2, output3]) 
model.compile(optimizer='adam', loss='mse')
print(model.summary())
# fit model
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 9)            0                                            
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 100)          1000        input_2[0][0]                    
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 1)            101         dense_13[0][0]                   
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 1)            101         dense_13[0][0]                   
____________________________________________________________________________________________

## Multi-step MLP Models
        -- here is little difference to the MLP model in predicting a vector output that represents different             output variables (as in the previous example) or a vector output that represents multiple time steps of one             variable.

### Multi-Step and Vector-output Model

In [25]:
# univariate multi-step vector-output mlp example
from numpy import array
from keras.models import Sequential 
from keras.layers import Dense

In [26]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out): 
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
# check if we are beyond the sequence 
        if out_end_ix > len(sequence):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix] 
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [27]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps_in)) 
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
print(model.summary())
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in))
yhat = model.predict(x_input, verbose=0)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 100)               400       
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 202       
Total params: 602
Trainable params: 602
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
print(yhat)

[[102.144135 112.65208 ]]


# Multivariate Multi-step MLP Models
        -- it may be a little more challenging, particularly in preparing the data and defining the shape of inputs and outputs for the model.
 
1. Multiple Input Multi-step Output.
2. Multiple Parallel Input and Multi-step Output.

## 1. Multiple Input Multi-step Output

In [29]:
########## Data Preparation #######

In [30]:

from numpy import array
from numpy import hstack
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out): 
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1 # check if we are beyond the dataset 
        if out_end_ix > len(sequences):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1] 
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [35]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90]) 
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))]) # convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1)) # horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq)) # choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out) 
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])
print(out_seq)

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]
[[ 25]
 [ 45]
 [ 65]
 [ 85]
 [105]
 [125]
 [145]
 [165]
 [185]]


In [34]:

# multivariate multi-step mlp example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input)) 
model.add(Dense(n_steps_out)) 
model.compile(optimizer='adam', loss='mse')
print(model.summary())
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 100)               700       
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 202       
Total params: 902
Trainable params: 902
Non-trainable params: 0
_________________________________________________________________
None
[[185.65697 206.56506]]


## 2. Multiple Parallel Input and Multi-step Output
        --A problem with parallel time series may require the prediction of multiple time steps of each time series.
            [[ 10 15 25] 
             [ 20 25 45] 
             [ 30 35 65] 
             [ 40 45 85] 
             [ 50 55 105] 
             [ 60 65 125] 
             [ 70 75 145] 
             [ 80 85 165] 
             [ 90 95 185]]
        
        
        input : 
                10, 15, 25
                20, 25, 45
                30, 35, 65
        output :
                40, 45, 85
                 50, 55, 105

In [36]:
# multivariate multi-step data preparation
from numpy import array
from numpy import hstack
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out): 
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
# check if we are beyond the dataset 
        if out_end_ix > len(sequences):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :] 
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out) 
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [37]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input)) # flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

In [38]:
X,y

(array([[ 10,  15,  25,  20,  25,  45,  30,  35,  65],
        [ 20,  25,  45,  30,  35,  65,  40,  45,  85],
        [ 30,  35,  65,  40,  45,  85,  50,  55, 105],
        [ 40,  45,  85,  50,  55, 105,  60,  65, 125],
        [ 50,  55, 105,  60,  65, 125,  70,  75, 145]]),
 array([[ 40,  45,  85,  50,  55, 105],
        [ 50,  55, 105,  60,  65, 125],
        [ 60,  65, 125,  70,  75, 145],
        [ 70,  75, 145,  80,  85, 165],
        [ 80,  85, 165,  90,  95, 185]]))

In [40]:
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
# flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input)) 
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')
print(model.summary())
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]]) 
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 606       
Total params: 1,606
Trainable params: 1,606
Non-trainable params: 0
_________________________________________________________________
None
[[ 91.214676  96.23492  187.02591  102.35888  106.570335 208.14363 ]]
